# Preprocessing CyGNSS data

Data is downloaded from NASA EarthCloud as described in the `APIs` notebook. For the expected format for CyGNSSnet, additional preprocessing steps are necessary.

In [10]:
import os
import sys
sys.path.append('../externals/gfz_cygnss/')

In [11]:
# !pip install tenacity

In [12]:
from gfz_202003.preprocessing import preprocess as prep

TypeError: <class 'numpy.typing._dtype_like._SupportsDType'> is not a generic class

In [ ]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
import seaborn as sns

import datetime
import xarray as xr

import argparse

In [7]:
import cdsapi

## Download raw CyGNSS data

The CyGNSSnet preprocessing routine expects the raw data files ordered as 

> `$raw_data_dir/<year>/<day-of-year>/cyg*.nc`

Data is always downloaded for one full day for all spacecraft, generating 8 `netcdf` files per day of observations. Below is a routine to specify a date range, followed by downloading the corresponding data and storing it in the appropriate subfolders.

In [6]:
raw_data_root = '/home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/raw_data'
dev_data_root = '/home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/dev_data'

Select a test day and prepared the input parameters for the provided download script

In [7]:
year  = 2021
month = 3
day   = 17

Downloaded target directory in the expected format `year/day-of-year`

In [8]:
raw_data_sub = datetime.datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d").strftime("%Y/%j")

raw_data_dir = os.path.join(raw_data_root, raw_data_sub)

print(raw_data_dir)

/home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/raw_data/2021/168


Start and end date of download range in the required format. The end date is midnight the next day, this way only the requested day's data is downloaded.

In [9]:
start_date = datetime.datetime(year, month, day).strftime("%Y-%m-%dT%H:%M:%SZ")
end_date   = datetime.datetime(year, month, day + 1).strftime("%Y-%m-%dT%H:%M:%SZ")

print(f'--start-date {start_date}')
print(f'--end-date   {end_date}')

--start-date 2021-06-17T00:00:00Z
--end-date   2021-06-18T00:00:00Z


In [ ]:
dday = datetime.datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d").strftime("%j") # need that later
dday

In [ ]:
%env PYTHONPATH=/home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/data-subscriber
!python /home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/data-subscriber/subscriber/podaac_data_downloader.py  -c CYGNSS_L1_V3.1 -d $raw_data_dir --start-date $start_date --end-date $end_date

env: PYTHONPATH=/home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/data-subscriber
[2022-09-12 16:00:57,433] {podaac_data_downloader.py:243} INFO - Found 7 total files to download
[2022-09-12 16:00:59,062] {podaac_access.py:446} WARNING - Computed checksum f11baba7acac4b5b14b3891e83f715c8 does not match expected checksum 10e4ef36d29f030ea7e524f8924389fc
[2022-09-12 16:01:46,860] {podaac_data_downloader.py:276} INFO - 2022-09-12 16:01:46.860919 SUCCESS: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/CYGNSS_L1_V3.1/cyg06.ddmi.s20210617-000000-e20210617-235959.l1.power-brcs.a31.d32.nc
[2022-09-12 16:01:48,483] {podaac_access.py:446} WARNING - Computed checksum 9b3100d23550d03cb85056609ecddd5b does not match expected checksum a8851840f3a4bbdc8499ea2f17d5119b
[2022-09-12 16:02:39,804] {podaac_data_downloader.py:276} INFO - 2022-09-12 16:02:39.804552 SUCCESS: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/CYGNSS_L1_V3.1/cyg08.ddmi.s2021061

## Download raw ERA5 data

The preprocessing pipeline requires the ERA5 windspeed labels. Download the raw ERA5 data for the same timespan.

In [1]:
era5_data = os.path.join(raw_data_dir, 'ERA5_windspeed.nc')

NameError: name 'os' is not defined

In [ ]:
cds = cdsapi.Client()

In [2]:
cds.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind',
        ],
        'year': year,
        'month': month,
        'day': day,
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00'
        ],
        'area': [
            40, -180, -40, 180,
        ],
    },
    era5_data)

NameError: name 'cds' is not defined

In [3]:
era5_ds = xr.open_dataset(era5_data)
era5_ds

NameError: name 'xr' is not defined

## Annotate raw CyGNSS data with windspeed labels

We need to create the data variables `ERA5_u10` and `ERA5_v10` and attach them to the CyGNSS raw data.

In [22]:
os.listdir(raw_data_dir)

['cyg07.ddmi.s20210617-000000-e20210617-235959.l1.power-brcs.a31.d32.nc',
 'ERA5_windspeed.nc',
 'cyg02.ddmi.s20210617-000000-e20210617-235959.l1.power-brcs.a31.d32.nc',
 'cyg04.ddmi.s20210617-000000-e20210617-235959.l1.power-brcs.a31.d32.nc',
 'cyg01.ddmi.s20210617-000000-e20210617-235959.l1.power-brcs.a31.d32.nc',
 'cyg06.ddmi.s20210617-000000-e20210617-235959.l1.power-brcs.a31.d32.nc',
 'CYGNSS_L1_V3.1.citation.txt',
 'cyg03.ddmi.s20210617-000000-e20210617-235959.l1.power-brcs.a31.d32.nc',
 'cyg08.ddmi.s20210617-000000-e20210617-235959.l1.power-brcs.a31.d32.nc']

Check units for spacetime coordinates
* Longitude
  * ERA5:   -180 ... 0 ... +180
  * CyGNSS: 0 ... 180 ... 360
* Latitude
  * ERA5 & CyGNSS: -40 ... 0 ... +40
* Timestamp


--> Need to shift the ERA5 longitude coordinate by 180

In [ ]:
def annotate_dataset(cygnss_file, era5_file, save_dataset=False):
    '''
    Annotate a given CyGNSS dataset with ERA5 windspeed labels and save to disk
    
    Parameters:
    cygnss_file : path to CyGNSS dataset
    era5_file   : path to orresponding ERA5 dataset
    save_dataset : if True, save dataset to disk overwriting cygnss_file (default: False)
    
    Returns:
    Annotated CyGNSS dataset
    '''
    
    # necessary because lazy loading prohibits overwriting the netcdf files at the end of this section
    with xr.open_dataset(cygnss_file) as data:
        cygnss_ds = data.load()
        
    with xr.open_dataset(era5_file) as data:
        era5_ds = data.load()
        
    # needs to be shifted by 180 for compatibility with CyGNSS
    era5_ds = era5_ds.assign_coords(longitude=era5_ds.coords['longitude'] + 180)
    
    interp_ds = era5_ds.interp(longitude=cygnss_ds.sp_lon, latitude=cygnss_ds.sp_lat, time=cygnss_ds.ddm_timestamp_utc)
    
    cygnss_ds['ERA5_u10'] = interp_ds['u10']
    cygnss_ds['ERA5_v10'] = interp_ds['v10']

    tmp_attrs = cygnss_ds['ERA5_u10'].attrs
    tmp_attrs['long_name'] = cygnss_ds['ERA5_u10'].long_name + ' (interpolated)'
    cygnss_ds['ERA5_u10'].attrs = tmp_attrs

    tmp_attrs = cygnss_ds['ERA5_v10'].attrs
    tmp_attrs['long_name'] = cygnss_ds['ERA5_v10'].long_name + ' (interpolated)'
    cygnss_ds['ERA5_v10'].attrs = tmp_attrs
    
    cygnss_ds = cygnss_ds.drop_vars(['longitude', 'latitude', 'time'])
    
    # dummy values only for preprocessing routine
    cygnss_ds['GPM_precipitation'] = -9999
    cygnss_ds['ERA5_mdts'] = -9999
    cygnss_ds['ERA5_mdww'] = -9999
    cygnss_ds['ERA5_swh'] = -9999
    cygnss_ds['ERA5_shts'] = -9999
    cygnss_ds['ERA5_shww'] = -9999
    cygnss_ds['ERA5_p140121'] = -9999
    cygnss_ds['ERA5_p140124'] = -9999
    cygnss_ds['ERA5_p140127'] = -9999
    
    if save_dataset:
        cygnss_ds.to_netcdf(cygnss_file)
        
    return cygnss_ds

In [ ]:
for cygnss_file in os.listdir(raw_data_dir):
    if cygnss_file.startswith('cyg') and cygnss_file.endswith('.nc'):
        print(cygnss_file)
        annotate_dataset(os.path.join(raw_data_dir, cygnss_file), era5_data, save_dataset=True)

## Check raw data

In [ ]:
from importlib import reload
reload(prep)
raw_ds = prep.open_mfdataset(os.path.join(raw_data_dir, cygnss_file))

raw_ds

In [ ]:
filtered_ds = prep.apply_quality_filter(raw_ds, is_ml_ops=True)
filtered_ds

In [ ]:
os.listdir('/work/ka1176/shared_data/2020-03/raw_data/2021/014/')

In [ ]:
bu = raw_ds['ddm_brcs_uncert']
qf = raw_ds['quality_flags']
st = raw_ds['nst_att_status']
fom = raw_ds['prn_fig_of_merit']
les = raw_ds['ddm_les']
rxg = raw_ds['sp_rx_gain']
nsca = raw_ds['nbrcs_scatter_area']
lsca = raw_ds['les_scatter_area']
lat = raw_ds['sp_lat']
lon = raw_ds['sp_lon']
ws = raw_ds['windspeed']

For now, use only the quality flag == 4

In [ ]:
quality = (bu<1) & (qf == 4) & (st == 0) & (fom > 3) & (rxg > 0) & (les >= 0)

In [ ]:
np.sum((bu<1) & (st==0)).compute()

## Created processed data

In [ ]:
raw_ds = prep.open_mfdataset(os.path.join(raw_data_dir, 'cyg06*.nc'), channels=[0,1,2,3])

In [ ]:
dev_data_dir = '/work/ka1176/shared_data/2022-cygnss-deployment/dev_data/'

In [ ]:
for ff in os.listdir('/work/ka1176/shared_data/2022-cygnss-deployment/raw_data/2021/168/'):
    tmp = xr.open_dataset(os.path.join('/work/ka1176/shared_data/2022-cygnss-deployment/raw_data/2021/168/', ff))
    if not 'ERA5_u10' in tmp.keys():
        print(ff)

In [ ]:
tmp

In [ ]:
reload(prep)
args = argparse.Namespace(raw_data_dir='/work/ka1176/shared_data/2022-cygnss-deployment/raw_data/',
                          output_dir=dev_data_dir,
                          v_map=['brcs'],
                          n_valid_days=0,
                          n_test_days=1,
                          n_processes=1,
                          only_merge=False,
                          use_land_data=False,
                          is_ml_ops=True,
                          version='v3.1',
                          day=dday,
                          year=year,
                          reduce_mode='')

prep.generate_input_data(args)

## Check the new CyGNSS data v3.1

In [ ]:
TODO annotate the samples with date (year month day etc)

In [ ]:
!conda list env